In [1]:
from ultralytics import YOLO
import cv2

import util
from sort.sort import *
from util import get_car, read_license_plate, write_csv

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Using CPU. Note: This module is much faster with a GPU.


In [16]:
  import PIL.Image
  if not hasattr(PIL.Image, 'Resampling'):  # Pillow<9.0
    PIL.Image.Resampling = PIL.Image
  # Now PIL.Image.Resampling.BICUBIC is always recognized.

In [2]:
results = {}

mot_tracker = Sort()

# load models
coco_model = YOLO('yolov8n.pt')
license_plate_detector = YOLO('/home/harshit/Desktop/project/license_plate_detector.pt')

# load video
cap = cv2.VideoCapture('/home/harshit/Desktop/project/sample.mp4')

In [17]:
vehicles = [2, 3, 5, 7]

# read frames
frame_nmr = -1
ret = True
while ret:
    frame_nmr += 1
    ret, frame = cap.read()
    if ret:
        results[frame_nmr] = {}
        # detect vehicles
        detections = coco_model(frame)[0]
        detections_ = []
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = detection
            
            if int(class_id) in vehicles:
                detections_.append([x1, y1, x2, y2, score])

        # track vehicles
        track_ids = mot_tracker.update(np.asarray(detections_))

        # detect license plates
        license_plates = license_plate_detector(frame)[0]
        for license_plate in license_plates.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = license_plate

            # assign license plate to car
            xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)

            if car_id != -1:

                # crop license plate
                license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]

                # process license plate
                license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
                _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV)

                # read license plate number
                license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop_thresh)

                if license_plate_text is not None:
                    results[frame_nmr][car_id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                                                  'license_plate': {'bbox': [x1, y1, x2, y2],
                                                                    'text': license_plate_text,
                                                                    'bbox_score': score,
                                                                    'text_score': license_plate_text_score}}
                    


0: 384x640 23 cars, 1 bus, 3 trucks, 14.4ms
Speed: 21.6ms preprocess, 14.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)



0: 384x640 2 0s, 7.1ms
Speed: 3.4ms preprocess, 7.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 cars, 1 bus, 2 trucks, 14.1ms
Speed: 7.0ms preprocess, 14.1ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 0s, 14.3ms
Speed: 5.3ms preprocess, 14.3ms inference, 3.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 2 trucks, 9.6ms
Speed: 4.8ms preprocess, 9.6ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 0s, 8.0ms
Speed: 3.6ms preprocess, 8.0ms inference, 3.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 2 trucks, 26.1ms
Speed: 7.1ms preprocess, 26.1ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 0s, 8.2ms
Speed: 3.8ms preprocess, 8.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 2 trucks, 16.5ms
Speed: 5.6ms preprocess, 16.5ms inference, 3.1ms p

In [18]:
write_csv(results,'/home/harshit/Desktop/project/test.csv')

{'car': {'bbox': [726.2982758598273, 1404.262137999619, 1420.7562534068252, 2043.2460946263172]}, 'license_plate': {'bbox': [956.6614379882812, 1815.88916015625, 1182.5322265625, 1907.11083984375], 'text': 'NA13NRU', 'bbox_score': 0.5408154129981995, 'text_score': 0.42209780558847443}}
{'car': {'bbox': [2192.415395123281, 1169.956506109836, 2809.748045074348, 1772.347502416672]}, 'license_plate': {'bbox': [2408.0185546875, 1577.60205078125, 2602.342529296875, 1658.579345703125], 'text': 'AU51YSU', 'bbox_score': 0.4979510009288788, 'text_score': 0.37632290149791847}}
{'car': {'bbox': [725.9480660478134, 1407.0738209739832, 1423.1740426825986, 2052.1907126157103]}, 'license_plate': {'bbox': [968.1912231445312, 1821.3585205078125, 1178.92333984375, 1899.4427490234375], 'text': 'NA13NRU', 'bbox_score': 0.5595883131027222, 'text_score': 0.7835059371218968}}
{'car': {'bbox': [723.0792403769201, 1408.5605084788626, 1418.8183110075024, 2053.383139450051]}, 'license_plate': {'bbox': [959.347961

In [1]:
results

NameError: name 'results' is not defined